# Resetting time to zero after cloning a climlab process

Brian Rose, 2/15/2022

Here are some notes on how to reset a model's internal clock to zero after cloning a process with `climlab.process_like()`

_These notes may become out of date after the next revision of climlab, because the calendar object that climlab uses will likely get replaced with something more robust in the future._

For posterity, this is the version of climlab we're using in this example:

In [1]:
import numpy as np
import climlab
climlab.__version__

'0.7.13'

## The climlab time dictionary

Every process object contains a `time` attribute, which is just a dictionary with various counters and information about timesteps.

Here we create a single-column radiation model `m1` with a timestep of 1 day, and inspect its `time` dictionary:

In [2]:
mystate = climlab.column_state()
m1 = climlab.radiation.RRTMG(state=mystate, timestep=climlab.utils.constants.seconds_per_day)
m1.time

{'timestep': 86400.0,
 'num_steps_per_year': 365.2422,
 'day_of_year_index': 0,
 'steps': 0,
 'days_elapsed': 0,
 'years_elapsed': 0,
 'days_of_year': array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
         11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
         22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
         33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
         44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
         55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
         66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
         77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
         88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
         99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
        110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
        121., 122., 123., 124., 125.

If we take a single time step forward, some elements in this dictionary get updated:

In [3]:
m1.step_forward()
m1.time

{'timestep': 86400.0,
 'num_steps_per_year': 365.2422,
 'day_of_year_index': 1,
 'steps': 1,
 'days_elapsed': 1.0,
 'years_elapsed': 0,
 'days_of_year': array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
         11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
         22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
         33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
         44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
         55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
         66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
         77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
         88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
         99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
        110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
        121., 122., 123., 124., 12

In particular, `steps` has increased by 1, and `days_elapsed` is now 1.0 (using a timestep of 1 day).

Let's now clone this model. Both the state and the calendar are cloned, so our new model has the same date as `m1`:

In [4]:
m2 = climlab.process_like(m1)
m2.time

{'timestep': 86400.0,
 'num_steps_per_year': 365.2422,
 'day_of_year_index': 1,
 'steps': 1,
 'days_elapsed': 1.0,
 'years_elapsed': 0,
 'days_of_year': array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
         11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
         22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
         33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
         44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
         55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
         66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
         77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
         88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
         99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
        110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
        121., 122., 123., 124., 12

## What if we want to clone the state, but reset the calendar to zero?

One simple hack is just to keep a copy of the initial `time` dictionary prior to taking any steps forward. 

We can do this with the `dict`'s built-in `copy()` method.

In [5]:
mystate2 = climlab.column_state()
m3 = climlab.radiation.RRTMG(state=mystate2, timestep=climlab.utils.constants.seconds_per_day)
zero_time = m3.time.copy()
m3.step_forward()
m4 = climlab.process_like(m3)
# Now both m3 and m4 have the same state:
assert m3.Ts == m4.Ts
assert np.all(m3.Tatm == m4.Tatm)
# And they also have the same calendar:
print('After cloning, m3 has taken {} steps, and m4 has taken {} steps.'.format(m3.time['steps'], m4.time['steps']))
# But we can reset the calendar for m4 as if it had never taken a step forward:
m4.time = zero_time
print('After replacing the time dict, m3 has taken {} steps, and m4 has taken {} steps.'.format(m3.time['steps'], m4.time['steps']))

After cloning, m3 has taken 1 steps, and m4 has taken 1 steps.
After replacing the time dict, m3 has taken 1 steps, and m4 has taken 0 steps.


Since we haven't changed any model parameters, they should both evolve exactly the same way on their next timestep so the states remain the same:

In [6]:
for model in [m3, m4]:
    model.step_forward()
assert m3.Ts == m4.Ts
assert np.all(m3.Tatm == m4.Tatm)
print('After one more step, m3 has taken {} steps, and m4 has taken {} steps.'.format(m3.time['steps'], m4.time['steps']))

After one more step, m3 has taken 2 steps, and m4 has taken 1 steps.


But if I now change a parameter in `m4`, their states will begin to differ:

In [7]:
m4.subprocess['SW'].S0 += 10.
for model in [m3, m4]:
    model.step_forward()
print('One step after changing S0 in m4, m3 has taken {} steps, and m4 has taken {} steps.'.format(m3.time['steps'], m4.time['steps']))
print('')
print('Now checking to see if the states are still the same:')
assert m3.Ts == m4.Ts

One step after changing S0 in m4, m3 has taken 3 steps, and m4 has taken 2 steps.

Now checking to see if the states are still the same:


AssertionError: 

The assertion fails because the surface temperatures of the two states have diverged, as expected.